# Text Mining Data Prep

### ISM6564

**Week04, Part01**

&copy; 2023 Dr. Tim Smith

<a target="_blank" href="https://colab.research.google.com/github/prof-tcsmith/ta-f23/blob/main/W04/4.1-Tutorial - text mining data prep fundamentals using sklearn.ipynb#offline=1">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---



# Introduction

In this notebook, we will prepare the data for text mining using techniques such as tokenization, stop word removal, and stemming. Also, we will represent our list of documents (in this case, a list of strings) as both a Count Vector and a Term Frequency Inverse Document (TF-ID) matrix. We will use the [scikit-learn](https://scikit-learn.org/stable/) library to perform these tasks.

In [1]:
import numpy as np
import pandas as pd
import re

# we will use spacy for lemmatization (it's much better than nltk)
import spacy

# we will use sklearn for feature extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# we will use sklearn for dimensionality reduction
from sklearn.decomposition import TruncatedSVD

let's start with a corpus

In [2]:
# Define the corpus of documents
corpus = [
    "This is the first document.",
    "This document is the second document.",
    "If he cares about caring, then he should care about caring about caring.",
    "If he began to care, then he should begin to care about caring about caring.",
    "123 the world is large 32.34",
    'He stripped the striped paint by stripping the first coat of paint.'
]


## Create a term by document matrix

TfidfVectorizer and CountVectorizer both are methods for converting text data into vectors as model can process only numerical data.

### Using CountVectorizer

In CountVectorizer we only count the number of times a word appears in the document which results in biasing in favour of most frequent words. this ends up in ignoring rare words which could have helped is in processing our data more efficiently.

In [3]:
# remove punctuation and numbers
corpus = [re.sub(r'[^a-zA-Z ]+', '', doc) for doc in corpus]

In [4]:
# CountVectorizer will covert to lowercase, remove punctuation, and remove stop words - to 
# remove other things, such as numbers, use the token_pattern parameter
vectorizer = CountVectorizer(stop_words='english', lowercase=True) # this will handle all text cleaning, except removing numbers
X = vectorizer.fit_transform(corpus)
df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df

,began,begin,care,cares,caring,coat,document,large,paint,second,striped,stripped,stripping,world
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,2,0,0,1,0,0,0,0
2,0,0,1,1,3,0,0,0,0,0,0,0,0,0
3,1,1,2,0,2,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,1
5,0,0,0,0,0,1,0,0,2,0,1,1,1,0


### Using TficVectorizer

To overcome this problem (over emphasis on high frequency), we use TfidfVectorizer .

In TfidfVectorizer we consider overall document weightage of a word. It helps us in dealing with most frequent words. Using it we can penalize them. TfidfVectorizer weights the word counts by a measure of how often they appear in the documents.

In [5]:
# Like CountVectorizer, TfidfVectorizer will covert to lowercase, remove punctuation, and remove 
# stop words - to remove other things, such as numbers, use the token_pattern parameter
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)

X = vectorizer.fit_transform(corpus)

df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df

,began,begin,care,cares,caring,coat,document,large,paint,second,striped,stripped,stripping,world
0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.8538,0.000000,0.000000,0.520601,0.000000,0.000000,0.000000,0.000000
2,0.00000,0.00000,0.295049,0.359809,0.885146,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.36812,0.36812,0.603728,0.000000,0.603728,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107
5,0.00000,0.00000,0.000000,0.000000,0.000000,0.353553,0.0000,0.000000,0.707107,0.000000,0.353553,0.353553,0.353553,0.000000


In [6]:
### Word Lemmatization

Notice that we might benefit from finding the lemma of a word. For example, the words "beginning", "begun", and "begins" are all related to the same concept or begin. We can use the NLTK's WordNetLemmatizer to reduce words to their lemmas.

> Stemming is a process that stems or removes last few characters from a word, often leading to incorrect meanings and spelling. Lemmatization considers the context and converts the word to its meaningful base form, which is called Lemma. For instance, stemming the word 'Caring' would return 'Car' whereas lemmatization would return 'Care'.

In [7]:

load_model = spacy.load("en_core_web_lg") # python -m spacy download en_core_web_lg

cleaned_corpus = []
for doc in corpus:
    doc = re.sub(r'[^a-zA-Z ]+', '', doc) # remove punctuation and numbers
    cleaned_corpus.append(" ".join([token.lemma_ for token in load_model(doc)]))
    
cleaned_corpus

['this be the first document',
 'this document be the second document',
 'if he care about care then he should care about care about care',
 'if he begin to care then he should begin to care about care about care',
 '  the world be large',
 'he strip the striped paint by strip the first coat of paint']

Now, let's use the TfidfVectorizer to convert our new lematized corpus into a matrix of TF-IDF features.

In [8]:
# Like CountVectorizer, TfidfVectorizer will covert to lowercase, remove punctuation, and remove 
# stop words - to remove other things, such as numbers, use the token_pattern parameter
vectorizer = TfidfVectorizer(token_pattern=r'[a-zA-Z]+', stop_words='english', lowercase=True)

X = vectorizer.fit_transform(cleaned_corpus)

df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df

,begin,care,coat,document,large,paint,second,strip,striped,world
0,0.000000,0.0000,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.0000,0.000000,0.8538,0.000000,0.000000,0.520601,0.000000,0.000000,0.000000
2,0.000000,1.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.520601,0.8538,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.0000,0.000000,0.0000,0.707107,0.000000,0.000000,0.000000,0.000000,0.707107
5,0.000000,0.0000,0.316228,0.0000,0.000000,0.632456,0.000000,0.632456,0.316228,0.000000


## Apply SVD for dimension reduction

Let's apply SVD to reduce the dimensionality of our data. 

> NOTE: Recall that the input dimensions will be the number of unique words in the corpus. With large corpa, the number of unique words can be very large, and thus the dimensionality of the data can be very large. Four our small corpus, the problem of high dimensionality not really a concern. With large corpo, you may need to reduce the dimensionality of the data to make it more manageable for the machine learning algorithms (especially clustering and neural networks).

In [10]:
#If you are performing Latent Semantic Analysis, recommended number of components is 100

svd = TruncatedSVD(n_components=5, n_iter=10)

In [11]:
X_svd = svd.fit_transform(X)
X_svd

array([[ 9.62756515e-01,  4.76282196e-17,  1.95021393e-19,
        -2.71812235e-20,  2.70369920e-01],
       [ 9.62756515e-01,  2.38913246e-16,  2.10643630e-18,
         9.84444962e-18, -2.70369920e-01],
       [-3.94316065e-16,  9.62756515e-01,  1.88520068e-16,
         2.74908048e-17,  1.88512648e-16],
       [-4.84059072e-16,  9.62756515e-01,  1.42434778e-16,
         1.97328598e-18,  2.98411134e-16],
       [-4.88512695e-18,  7.57433136e-17,  5.06014611e-01,
         8.62524906e-01, -1.30008916e-17],
       [ 9.34913517e-19,  1.71677325e-16,  8.62524906e-01,
        -5.06014611e-01,  8.37407030e-18]])

In [12]:
X_svd.shape[1]

5

In [13]:
df = pd.DataFrame(X_svd, columns=[f"svd{num:04}" for num in range(0,X_svd.shape[1])])
df

,svd0000,svd0001,svd0002,svd0003,svd0004
0,9.627565e-01,4.762822e-17,1.950214e-19,-2.718122e-20,2.703699e-01
1,9.627565e-01,2.389132e-16,2.106436e-18,9.844450e-18,-2.703699e-01
2,-3.943161e-16,9.627565e-01,1.885201e-16,2.749080e-17,1.885126e-16
3,-4.840591e-16,9.627565e-01,1.424348e-16,1.973286e-18,2.984111e-16
4,-4.885127e-18,7.574331e-17,5.060146e-01,8.625249e-01,-1.300089e-17
5,9.349135e-19,1.716773e-16,8.625249e-01,-5.060146e-01,8.374070e-18


As you can see, we have taken the 14 dimensions of input and reduced these down to 5 dimensions. This is a 64% reduction in the number of dimensions. 

### Now we are ready to use this data as input 

Our text data is now ready to be used in a model. If we have other classification meta data (for instance, news category, or customer or not, etc.), we can create predictive models using machine learning techniques. If we don't have any tags/etadata, we can use this data to cluster the documents - or, go through the documents manually and tag them.